In [1]:
import torch
from transformers import BertTokenizerFast,BertModel
from data.call_func import _get_tokenizer
from dotted_dict import DottedDict
import torch
import torch.optim
import torch.utils.data
from data.call_func import sighan_error_correct_prompt,sighan_error_detec_csc

from data.data_entry import get_loader, get_dataset
from model.model_entry import select_model

from utils.args_util import load_args
from trainers.train_corrector import TrainerCorrector
from optimizer import get_optimizer
from data.utils import build_candidates_dict
import os
from collections import defaultdict
from metrics import compute_csc_metrics_by_file
torch.set_printoptions(precision=2)
from data.utils import word2pinyin_similar
from data.call_func import _get_tokenizer
from itertools import product
import numpy as np

#加载训练参数
config_yaml = "test_csc.yaml"
args = load_args("./configs/%s"%config_yaml)
train_dataset = get_dataset(args.dataset.train)
test_dataset = get_dataset(args.dataset.test)
_tokenizer = _get_tokenizer()

D:\论文代码和数据备份\CSC\utils\args_util.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  args = DottedDict(yaml.load(open(yaml_path, "r"), Loader=None))
D:\论文代码和数据备份\CSC\utils\args_util.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  args = DottedDict(yaml.load(open(yaml_path, "r"), Loader=None))


D:\论文代码和数据备份\CSC
279379 samples found in ./resources/merged/clean_extend_training_input_shuf_v1.txt
D:\论文代码和数据备份\CSC
1100 samples found in ./resources/sighan15/TestInput.txt
load saved tokens


C:\ProgramData\Anaconda3\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [2]:
#加载detec模型
device = "cpu"
config_yaml = "test_csc.yaml"
args = load_args("./configs/%s"%config_yaml)
detec_model = select_model(args.model).to(device)
detec_model.encoder.resize_token_embeddings(len(_tokenizer))

D:\论文代码和数据备份\CSC\model\copy_transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mode

Embedding(21954, 768)

In [3]:
from transformers import  BertTokenizer
len(BertTokenizer.from_pretrained("bert-base-chinese"))

21128

In [4]:
def my_tokenize(self, text):
    # print("use my tokenizer !!!")
    return list(text.lower())
import types
from transformers import BertTokenizer
BertTokenizer.my_tokenize = my_tokenize

# detec_model.load_state_dict(torch.load("./saved/pinyin_corrector/20211104150912/model_epoch18_f10.772646531414744.pkl",map_location="cpu")['model_state_dict'])
detec_model.load_state_dict(torch.load("./saved/pinyin_corrector/20211104150721/model_epoch4_f10.767172162709159.pkl",map_location="cpu")['model_state_dict'])

<All keys matched successfully>

In [5]:
word2candidates = build_candidates_dict()

瓠互沪楛沍护扈	壶胡鹄蝴胡琥湖葫囫弧虎许斛瑚惚唬乎戏呼糊忽狐	富腹馥覆赙附副傅复付复赴妇咐父阜赋负驸讣	氟缚匐肤𫓧俯府腑夫腐𫖯蝠绋扶甫幅芾孚弗辐绂福袱浮伏符菔釜敷拂涪抚怫斧孵莆趺辅佛服父麸脯芙伕俘


In [6]:
def build_shape_candidates_dict():
    #取得一个汉字的近形字和近音字
    word2candidates = defaultdict(set)
    with open(r"resources/gcn_graph.ty_xj/SimilarShape_simplied.txt","r",encoding="utf-8") as f:
        lines = f.readlines()[1:]
        for l in  lines:
            w,candiates = [x.strip() for x in l.split(",",1)]
            word2candidates[w].update(candiates)
    return word2candidates

word2shape_candidates = build_shape_candidates_dict()

In [7]:
def add_candidates(text_file,label_file):
    w2c = defaultdict(set)
    f1,f2 = open(text_file,"r",encoding="utf-8"),open(label_file,"r",encoding="utf-8")
    for text,label in zip(list(f1),list(f2)):
        text = text.split("\t")[1].strip()
        label_splits = label.strip().split(",")
        if label_splits[-1] == "":
            del label_splits[-1]
        if len(label_splits) < 3:
            continue
        for i in range(1,len(label_splits),2):
            pos,candidate = label_splits[i],label_splits[i+1]
            idx = int(pos) - 1
            w2c[text[idx]].add(candidate.strip())
    return w2c
#从训练集中提取候选集
add_w2c = add_candidates("./resources/merged/clean_extend_training_input_shuf_v1.txt","./resources/merged/clean_extend_training_truth_shuf_v1.txt")

In [8]:
print(add_w2c["地"])

{'堤', '在', '底', '得', '递', '低', '迪', '滴', '多', '的', '弟', '帝', '第', '抵'}


In [9]:
def step(data):
        inputs = data
        for k in inputs.keys():
            if isinstance(inputs[k], torch.Tensor):
                inputs[k] = inputs[k].to(device)
        return inputs

In [12]:
def write_csc_predictions( input_ids, predict_labels, sentence_ids, save_path, is_13 = False):
    tokenizer = _get_tokenizer()
    comma_token_id = tokenizer.convert_tokens_to_ids([","])[0]
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    with open(save_path, "w", encoding="utf-8") as f:
        # print(len(input_ids),len(predict_labels),len(sentence_ids))
        punctuations = ['"', '”', ">", "》", "]", "】"]
        for tl,pl,sid in zip(input_ids,predict_labels,sentence_ids):
            corrected = [sid]
            for i in range(len(tl)):
                if tl[i] == tokenizer.sep_token_id:
                    break
                if tl[i] == pl[i]:
                    continue
                pred_token = tokenizer.convert_ids_to_tokens([pl[i]])[0].strip()
                if len(pred_token) != 1 or pred_token in punctuations:
                    continue
                true_token = tokenizer.convert_ids_to_tokens( tl[i] )[0].strip()
                if pred_token not in word2candidates[true_token] and true_token not in word2candidates[pred_token] and true_token not in add_w2c[pred_token] and pred_token not in add_w2c[true_token] and word2pinyin_similar(true_token) != word2pinyin_similar(pred_token):
#                     print(pred_token,true_token)
                    continue
                if pred_token in "得的地" and is_13: #remove "的得地" in sighan13
                    continue
                corrected.extend([i, pred_token])
            if len(corrected) == 0:
                corrected.append(0)
            corrected_str = ", ".join([str(c) for c in corrected]) + "\n"
            f.write(corrected_str)
            
def get_candidates_all(w):
    results = []
    results += list(word2candidates[w]) + list(add_w2c[w])
    return list (set(results))

from collections import Counter
from tqdm import tqdm_notebook
#统计词频
wcnt = Counter()
with open("./resources/merged/clean_extend_training_input_shuf_v1.txt","r",encoding="utf-8") as f:
    for l in tqdm_notebook(list(f)):
        for i in range(len(l) - 1):
            wcnt.update([l[i:i+2]])
            

#使用候选集框定正确的汉字
def predict(text,use_pinyin=True):
        id = ""
        label = ""
        raw_item =(id,text,label)
        inputs = sighan_error_detec_csc([raw_item])
        if not use_pinyin:
            inputs['pinyin_ids'] = None
        inputs = step(inputs)
        detec_logits = detec_model(inputs)
        return detec_logits.softmax(dim=2)
def get_candidates(w):
#     print("retrieve candidates:", w)
    tokens = list(set(list(word2candidates[w] )+list(add_w2c[w])))
    ids = torch.LongTensor(tokenizer.convert_tokens_to_ids(tokens)).to(device)
    return tokens,ids
def get_err_logit(logits, predict_text, original_text):
    logit = 1
    similarity = 0
    cnt = 0
    for i in range(len(original_text)):
        oc = original_text[i]
        pc = predict_text[i]
        if oc == pc:
            continue
        logit *= logits[i+1][tokenizer.convert_tokens_to_ids([pc])[0]].item()
        cnt += 1
        similarity += get_similar_score(oc,pc)
    return logit, similarity


def get_similar_score(a,b):
    def clean_pinyin(x):
        return x.replace("an","a").replace("en","e").replace("on","o").replace("in","i")
    a = a.replace("地","的")
    b =  b.replace("地","的")
    pinyins_a = [clean_pinyin (word2pinyin_similar(x) ) for x in a]
    pinyins_b = [clean_pinyin(word2pinyin_similar(x)) for x in b]
    pinyin_score = 0
    for p in pinyins_a:
        if p in pinyins_b:
            pinyin_score += 1
    score = pinyin_score 
    
    shape_score = 0
    for x,y in zip(a,b):
        if x in word2shape_candidates[y] or y in word2shape_candidates[x]:
            shape_score += 1
    score += shape_score
    return score

def is_good_text(text):
    logits = predict(text)
    ids = logits.argmax(dim=2)
    token_logits = logits.gather(2,ids[:,:,None]).squeeze()
    if token_logits.min() < 0.5:
        return False
    return True

def get_top_tokens(t, l,ids):
    return np.array(t)[l[ids].argsort(descending=True).cpu().numpy()][:5].tolist()

def get_top_tokens_by_pinyin(t,l,ids, pinyin_code):
    lst =  np.array(t)[l[ids].argsort(descending=True).cpu().numpy()].tolist()
    lst1 = [c for c in lst if word2pinyin_similar(c) == pinyin_code]
    lst2 = [c for c in lst if word2pinyin_similar(c) != pinyin_code]
    return lst1[:3] + lst2[:3]

def possible_err_positions(logits):
    ids = logits.argmax(dim=2)
    token_logits = logits.gather(2,ids[:,:,None]).squeeze()
#     print(token_logits)
    tensor_ids = torch.where( token_logits < 0.8 )[0]
    if len(tensor_ids) == 1:
        device = tensor_ids.device
        tensor_ids = tensor_ids.item()
        if token_logits[tensor_ids-1]   <  0.95 and (token_logits[tensor_ids-1] * token_logits[tensor_ids]) < 0.64:
            tensor_ids = torch.LongTensor([tensor_ids-1,(tensor_ids)]).to(device)
        elif  token_logits[tensor_ids+1] < 0.95 and (token_logits[tensor_ids] * token_logits[tensor_ids+1]) < 0.64:
            tensor_ids = torch.LongTensor([tensor_ids,(tensor_ids+1)]).to(device)
        else:
            tensor_ids =  torch.LongTensor([tensor_ids]).to(device)
    return  tensor_ids 

def is_bi_err(tensor_ids):
    ids =tensor_ids.tolist()
    return len(ids)==2 and ids[0] + 1== ids[1]


            
            
            
        
    

def solve_bi_err(index,logits,original_text, target=0.9):
    detec_predicts = logits.argmax(dim=2)
    predict_text = "".join(tokenizer.convert_ids_to_tokens(detec_predicts[0][1:-1]))
    print("predict_text:",predict_text)
    tokens = list(predict_text)
    l1,l2 = logits[0][index]
    p1,p2 = index[0]-1, index[1]-1
    

    t1,ids1 = get_candidates(original_text[index[0]-1])
    t2,ids2 = get_candidates(original_text[index[1]-1])
    
#     t1_pinyin, t1_other = [c for c in t1 if word2pinyin_similar(c) == word2pinyin_similar(original_text[index[0]-1])],  \ 
#     [c for c in t1 if word2pinyin_similar(c) != word2pinyin_similar(original_text[index[0]-1])]
    
#     t2_pinyin, t2_other = [c for c in t2 if word2pinyin_similar(c) == word2pinyin_similar(original_text[index[1]-1])],  \ 
#     [c for c in t2 if word2pinyin_similar(c) != word2pinyin_similar(original_text[index[1]-1])]
    c1 =[original_text[p1],predict_text[p1]]  + get_top_tokens_by_pinyin(t1,l1,ids1,word2pinyin_similar(original_text[p1]))
    c2 =[original_text[p2],predict_text[p2]] + get_top_tokens_by_pinyin(t2,l2,ids2,word2pinyin_similar(original_text[p2]))
#     c1 =[predict_text[p1]] + get_top_tokens(t1,l1,ids1)
#     c2 =[predict_text[p2]] + get_top_tokens(t2,l2,ids2)
    print(c1,c2)
    correct_predict_ids = None
    correct_text = None
    max_logit = 0
    similar_score = 0
    total_score = 0
    
    absolute_good_candidates = []
    for s1,s2 in list(product(c1,c2)):
        #原则：优先选概率高和相似度高的
#         wf = min([wcnt[x] for x in wcnt])
#         print( s1+s2 ,predict_text[p1]+predict_text[p2],wcnt[s1+s2])
        if wcnt[s1+s2] < 3 and s1+s2 != predict_text[p1]+predict_text[p2]:
            continue
        
        tokens[p1] = s1
        tokens[p2] = s2
        text = "".join(tokens)
        logits = predict(text, use_pinyin=True) 
        ids = logits.argmax(dim=2)
        token_logits = logits.gather(2,ids[:,:,None]).squeeze()
        new_text = "".join(tokenizer.convert_ids_to_tokens(ids.cpu().tolist()[0])[1:-1])
        no_deep_modify = text==new_text#text[:p1]==new_text[:p1] and text[p2+1]==new_text[p2+1]
#         print(s1+s2,no_deep_modify,text,new_text)

        if no_deep_modify :
            reselect_score = 0
            new_similar_score = get_similar_score(original_text[p1:p2+1],s1+s2)

            reselect_score += new_similar_score
            if wcnt[s1+s2] > 1000:
                reselect_score +=1
            new_logit = token_logits[p1+1] * token_logits[p2+1]
            if new_logit> target:
                absolute_good_candidates.append((new_text, new_logit.item(),s1+s2, reselect_score,wcnt[s1+s2],  ids.cpu().numpy().tolist()))
    print(correct_text,max_logit)

    if len(absolute_good_candidates) == 0:
        return detec_predicts
    else:
        absolute_good_candidates = sorted(absolute_good_candidates,key=lambda x: (-x[3], -x[1]))
    print(absolute_good_candidates[0][:5])
    return absolute_good_candidates[0][-1]
        



def check_text(text, poses):
    logits = predict(predict_text)
    ids = logits.argmax(dim=2)
    token_logits = logits.gather(2,ids[:,:,None]).squeeze()
    min_logit = min([token_logits[i+1] for i in poses])
    return min_logit
    
def solve_single_err(index,logits,original_text):
    
    detec_predicts = logits.argmax(dim=2)
    predict_text = "".join(tokenizer.convert_ids_to_tokens(detec_predicts[0][1:-1]))
    tokens = list(predict_text)
    
    l1 = logits[0][index][0]
    p1 = index[0]-1
    #确认修改后的汉字没问题的话，直接返回
#     min_logit = check_text(predict_text,[p1])
# #     print("min_logit:",min_logit)
#     if min_logit > 0.5:
#         return None, None
#     else:
#         print(predict_text[p1],":",min_logit.item())
    if original_text[p1] in get_candidates_all(predict_text[p1]) or predict_text[p1] in get_candidates_all(original_text[p1]) or predict_text[p1] == original_text[p1]:
        return None, None
    
    t1, ids1 = get_candidates(original_text[p1])
    c1 =[original_text[p1]] + [ predict_text[p1]] + get_top_tokens(t1,l1,ids1)
    
    print("predict_text:",predict_text, "err:",predict_text[p1] )

    logit = 0
    similar_score = 0
    logit = 0
    correct_text = None
    correct_ids = None
    
    all_candidate_sentence = []
    for s1 in c1:
        tokens[p1] = s1
        text = "".join(tokens)
        logits = predict(text)
        ids = logits.argmax(dim=2)
        token_logits = logits.gather(2,ids[:,:,None]).squeeze()
        new_text = "".join(tokenizer.convert_ids_to_tokens(ids.cpu().tolist()[0])[1:-1])
        span = original_text[p1-1:p1+2]
        new_span = new_text[p1-1:p1+2]
        cnt = 0

            
#         if span[0] != new_span[0] or span[-1] !=new_span[-1]:
            
#             no_deep_modify = text[p1]==new_text[p1] 
        new_similar_score = get_similar_score(span,new_span)
        print(span,new_span, get_similar_score(span,new_span))
#         for s,b in zip(span,new_span):
#             if a!=b:
#                 cnt += 1
#         if cnt <=1:
#             print(cnt)
#             continue
        new_logit = token_logits[p1+1]
#         print(s1, "%.2f"%new_logit.item(),new_text,new_similar_score)
        if new_logit > logit and new_similar_score >= similar_score:
            correct_text, logit, similar_score,correct_ids = new_text, new_logit, new_similar_score,ids
        elif new_logit - logit > - 0.01 and new_similar_score > similar_score:
            correct_text, logit, similar_score,correct_ids = new_text, new_logit, new_similar_score,ids
            
    return correct_text,correct_ids

def check_continue_err(text1,text2):
    errs = []
    for i in range(len(text1)-1):
        if text1[i] != text2[i] and text1[i+1] != text2[i+1]:
            return [i+1, i+2]
    return []

<ipython-input-12-4dd7f5d99b19>:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm_notebook(list(f)):


  0%|          | 0/279379 [00:00<?, ?it/s]

In [13]:
#sighan15
tokenizer = _get_tokenizer()
predict_labels = []
input_ids = []
sentence_ids = []
sig = "15"
test_data_arg = args.dataset.test
test_data_arg.text_path = "./resources/sighan{}/TestInput.txt".format(sig)
test_data_arg.label_path = "./resources/sighan{}/TestTruth.txt".format(sig)

test_dataset = get_dataset(test_data_arg)
from tqdm  import tqdm_notebook
with torch.no_grad():
    for i in tqdm_notebook(range(len(test_dataset))):
        raw_item = test_dataset.__getitem__(i)
        original_text = raw_item[1]
        inputs = sighan_error_detec_csc([raw_item])
        inputs = step(inputs)
        detec_logits = detec_model(inputs)
        detec_predicts = detec_logits.argmax(dim=2).cpu().numpy().tolist()
        detec_tokens = tokenizer.convert_ids_to_tokens(detec_predicts[0])[1:-1]
        
        
        #对连续错误进行处理
        predict_text = "".join(detec_tokens)
        logits = detec_logits.softmax(dim=2)
        tensor_ids = possible_err_positions(logits)


        new_error_outer = False
        for i in range(len(original_text) -1):
            if original_text[i] != predict_text[i] and original_text[i+1] != predict_text[i+1] and (predict_text[i] not in get_candidates(original_text[i])[0]  or predict_text[i+1] not in get_candidates(original_text[i+1])[0]):
                import re
                if len(re.findall(r'([\u2E80-\u9FFF]+)',original_text[i] )) == 0:
                    continue
                new_error_outer = True
                tensor_ids = torch.LongTensor([i+1,i+2]).to(tensor_ids.device)
                break
                
        target = 0.9

        if len(tensor_ids) == 2 and tensor_ids[0]+ 1 == tensor_ids[1] or new_error_outer:
            print("开始处理连续错误：", raw_item[1])
            modified_predict_ids = solve_bi_err(tensor_ids,logits,raw_item[1],target)
            if modified_predict_ids is not None:
                detec_predicts = modified_predict_ids
            print()
        
            
        tokenizer = _get_tokenizer()
        
        input_ids.extend(inputs['input_ids'].detach().cpu().numpy().tolist())
  

        predict_labels.extend(detec_predicts)
        
        sentence_ids.extend(inputs['sentence_ids'])
        
        predict_text = "".join(tokenizer.convert_ids_to_tokens(detec_predicts[0][1:-1]))


save_path = "../test_file"
write_csc_predictions(input_ids, predict_labels, sentence_ids, save_path)
metrics = compute_csc_metrics_by_file(save_path,"./resources/sighan{}/TestTruth.txt".format(sig), show=True)

D:\论文代码和数据备份\CSC
1100 samples found in ./resources/sighan15/TestInput.txt


<ipython-input-13-76d0a40e71fd>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(test_dataset))):


  0%|          | 0/1100 [00:00<?, ?it/s]

开始处理连续错误： 我以前想要高诉你，可是我忘了。我真户秃。
predict_text: 我以前想要告诉你，可是我忘了。我真感歉。
['户', '感', '胡', '糊', '狐', '尸', '富', '敷'] ['秃', '歉', '涂', '突', '吐', '秀', '妒', '稚']
None 0
('我以前想要告诉你，可是我忘了。我真糊涂。', 0.999961256980896, '糊涂', 2, 23)

开始处理连续错误： 上年春天我到台湾东部到台东去看到我的好朋友，因为我们面件很久。
predict_text: 上年春天我到台湾东部到台东去看到我的好朋友，因为我们面见很久。
['面', '面', '面', '沔', '偭', '们', '夤', '栘'] ['件', '见', '见', '肩', '间', '签', '价', '鲜']
None 0
('上年春天我到台湾东部到台东去看到我的好朋友，因为我们面见很久。', 0.9993343949317932, '面见', 3, 11)

开始处理连续错误： 可以问路的女人是他的是新中文老师。
predict_text: 可以问路的女人是他的是新中文老师。
['是', '是', '实', '世', '师', '摄', '次', '定'] ['新', '新', '心', '信', '姓', '乡', '津', '敬']
None 0

开始处理连续错误： 你可以告诉我那家书店有中文数马？
predict_text: 你可以告诉我那家书店有中文课吗？
['数', '课', '书', '输', '素', '硕', '锁', '说'] ['马', '吗', '吗', '嘛', '码', '么', '验', '𫘪']
None 0
('你可以告诉我那家书店有中文书吗？', 0.9985228776931763, '书吗', 3, 4)

开始处理连续错误： 可是很多外国人很喜欢赛太杨在海边，在印尼有很多外国人来路性。
predict_text: 可是很多外国人很喜欢晒太阳在海边，在印尼有很多外国人来旅行。
['路', '旅', '旅', '露', '绿', '女', '跳', '度'] ['性', '行', '行', '信', '兴', '金', '精', '牛']
None 0
('可是很多外国人很喜欢晒太

In [14]:
#sighan14
tokenizer = _get_tokenizer()
predict_labels = []
input_ids = []
sentence_ids = []
sig = "14"
test_data_arg = args.dataset.test
test_data_arg.text_path = "./resources/sighan{}/TestInput.txt".format(sig)
test_data_arg.label_path = "./resources/sighan{}/TestTruth.txt".format(sig)

test_dataset = get_dataset(test_data_arg)
from tqdm  import tqdm_notebook
with torch.no_grad():
    for i in tqdm_notebook(range(len(test_dataset))):
        raw_item = test_dataset.__getitem__(i)
        original_text = raw_item[1]
        inputs = sighan_error_detec_csc([raw_item])
        inputs = step(inputs)
        detec_logits = detec_model(inputs)
        detec_predicts = detec_logits.argmax(dim=2).cpu().numpy().tolist()
        detec_tokens = tokenizer.convert_ids_to_tokens(detec_predicts[0])[1:-1]
        
        
        #对连续错误进行处理
        predict_text = "".join(detec_tokens)
        logits = detec_logits.softmax(dim=2)
        tensor_ids = possible_err_positions(logits)


        new_error_outer = False
        for i in range(len(original_text) -1):
            if original_text[i] != predict_text[i] and original_text[i+1] != predict_text[i+1] and (predict_text[i] not in get_candidates(original_text[i])[0]  or predict_text[i+1] not in get_candidates(original_text[i+1])[0]):
                import re
                if len(re.findall(r'([\u2E80-\u9FFF]+)',original_text[i] )) == 0:
                    continue
                new_error_outer = True
                tensor_ids = torch.LongTensor([i+1,i+2]).to(tensor_ids.device)
                break
                
        target = 0.9

        if len(tensor_ids) == 2 and tensor_ids[0]+ 1 == tensor_ids[1] or new_error_outer:
            print("开始处理连续错误：", raw_item[1])
            modified_predict_ids = solve_bi_err(tensor_ids,logits,raw_item[1],target)
            if modified_predict_ids is not None:
                detec_predicts = modified_predict_ids
            print()
        
            
        tokenizer = _get_tokenizer()
        
        input_ids.extend(inputs['input_ids'].detach().cpu().numpy().tolist())
  

        predict_labels.extend(detec_predicts)
        
        sentence_ids.extend(inputs['sentence_ids'])
        
        predict_text = "".join(tokenizer.convert_ids_to_tokens(detec_predicts[0][1:-1]))


save_path = "../test_file"
write_csc_predictions(input_ids, predict_labels, sentence_ids, save_path)
metrics = compute_csc_metrics_by_file(save_path,"./resources/sighan{}/TestTruth.txt".format(sig), show=True)

D:\论文代码和数据备份\CSC
1062 samples found in ./resources/sighan14/TestInput.txt


<ipython-input-14-ef40241649e9>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(test_dataset))):


  0%|          | 0/1062 [00:00<?, ?it/s]

开始处理连续错误： 在次，我想最重要的还是能于培养出一群杰出的资源，能于为国家争荣、愿意贡献一份力量的人才。
predict_text: 在此，我想最重要的还是能于培养出一群杰出的资源，能于为国家争荣、愿意贡献一份力量的人才。
['在', '在', '再', '载', '宰', '台', '才', '子'] ['次', '此', '此', '刺', '魑', '思', '款', '子']
None 0
('在此，我想最重要的还是能于培养出一群杰出的资源，能于为国家争荣、愿意贡献一份力量的人才。', 0.9973002672195435, '在此', 3, 1623)

开始处理连续错误： 因为妈妈是给我出生的人，养我从小到大，她找钱得很辛苦，为了养小孩，照顾孩子，给小孩读书毕业，现在我作工了我更体会她的劳苦劳累。
predict_text: 因为妈妈是给我出生的人，养我从小到大，她赚钱得很辛苦，为了养小孩，照顾孩子，给小孩读书毕业，现在我做工了我更体会她的劳苦劳累。
['作', '做', '做', '坐', '佐', '任', '你', '非'] ['工', '工', '功', '公', '供', '空', '红', '害']
None 0
('因为妈妈是给我出生的人，养我从小到大，她赚钱得很辛苦，为了养小孩，照顾孩子，给小孩读书毕业，现在我做功了我更体会她的劳苦劳累。', 0.9966109395027161, '做功', 3, 16)

开始处理连续错误： 虽然这个世界每天都有人当妈妈，但真的将妈妈的角色做得好，不是每个人能成功做得到。「妈妈」不尽是个名子，也需要行动配著他。
predict_text: 虽然这个世界每天都有人当妈妈，但真的将妈妈的角色做得好，不是每个人能成功做得到。「妈妈」不仅是个名字，也需要行动配合她。
['著', '合', '注', '煮', '诸', '照', '作', '佐'] ['他', '她', '她', '他', '它', '地', '大', '人']
None 0
('虽然这个世界每天都有人当妈妈，但真的将妈妈的角色做得好，不是每个人能成功做得到。「妈妈」不仅是个名字，也需要行动配合她。', 0.9997900128364563, '合她', 2, 3)

开始处理连续错误： 如果，没有她这么努力的一至帮我，今天我的身活就会改变

In [15]:
#sighan13
tokenizer = _get_tokenizer()
predict_labels = []
input_ids = []
sentence_ids = []
sig = "13"
test_data_arg = args.dataset.test
test_data_arg.text_path = "./resources/sighan{}/TestInput.txt".format(sig)
test_data_arg.label_path = "./resources/sighan{}/TestTruth.txt".format(sig)

test_dataset = get_dataset(test_data_arg)
from tqdm  import tqdm_notebook
with torch.no_grad():
    for i in tqdm_notebook(range(len(test_dataset))):
        raw_item = test_dataset.__getitem__(i)
        original_text = raw_item[1]
        inputs = sighan_error_detec_csc([raw_item])
        inputs = step(inputs)
        detec_logits = detec_model(inputs)
        detec_predicts = detec_logits.argmax(dim=2).cpu().numpy().tolist()
        detec_tokens = tokenizer.convert_ids_to_tokens(detec_predicts[0])[1:-1]
        
        
        #对连续错误进行处理
        predict_text = "".join(detec_tokens)
        logits = detec_logits.softmax(dim=2)
        tensor_ids = possible_err_positions(logits)


        new_error_outer = False
        for i in range(len(original_text) -1):
            if original_text[i] != predict_text[i] and original_text[i+1] != predict_text[i+1] and (predict_text[i] not in get_candidates(original_text[i])[0]  or predict_text[i+1] not in get_candidates(original_text[i+1])[0]):
                import re
                if len(re.findall(r'([\u2E80-\u9FFF]+)',original_text[i] )) == 0:
                    continue
                new_error_outer = True
                tensor_ids = torch.LongTensor([i+1,i+2]).to(tensor_ids.device)
                break
                
        target = 0.9

        if len(tensor_ids) == 2 and tensor_ids[0]+ 1 == tensor_ids[1] or new_error_outer:
            print("开始处理连续错误：", raw_item[1])
            modified_predict_ids = solve_bi_err(tensor_ids,logits,raw_item[1],target)
            if modified_predict_ids is not None:
                detec_predicts = modified_predict_ids
            print()
        
            
        tokenizer = _get_tokenizer()
        
        input_ids.extend(inputs['input_ids'].detach().cpu().numpy().tolist())
  

        predict_labels.extend(detec_predicts)
        
        sentence_ids.extend(inputs['sentence_ids'])
        
        predict_text = "".join(tokenizer.convert_ids_to_tokens(detec_predicts[0][1:-1]))


save_path = "../test_file"
write_csc_predictions(input_ids, predict_labels, sentence_ids, save_path,is_13=True)
metrics = compute_csc_metrics_by_file(save_path,"./resources/sighan{}/TestTruth.txt".format(sig), show=True)

D:\论文代码和数据备份\CSC
998 samples found in ./resources/sighan13/TestInput.txt


<ipython-input-15-a522bc362d05>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(test_dataset))):


  0%|          | 0/998 [00:00<?, ?it/s]

开始处理连续错误： 爱迪生，小时候被所有人骂他是低能儿，可是他永有不屈不侥像上的经神，从火海中在站起来，后来他成为了不起的发明家。
predict_text: 爱迪生，小时候被所有人骂他是低能儿，可是他拥有不屈不屈向上的精神，从火海中再站起来，后来他成为了不起的发明家。
['侥', '屈', '骄', '交', '娇', '肖', '朽', '挠'] ['像', '向', '向', '像', '想', '心', '强', '信']
None 0
('爱迪生，小时候被所有人骂他是低能儿，可是他拥有不屈不肖像上的精神，从火海中再站起来，后来他成为了不起的发明家。', 0.9984805583953857, '肖像', 2, 47)

开始处理连续错误： 身为学生，教室是最好的游乐场，因为一天可以利用的时间一半都在教室，从桌椅到门窗，从黑板到垃圾桶，都可以把完、欣赏。
predict_text: 身为学生，教室是最好的游乐场，因为一天可以利用的时间一半都在教室，从桌椅到门窗，从黑板到垃圾桶，都可以玩玩、欣赏。
['把', '玩', '吧', '八', '巴', '办', '抓', '犯'] ['完', '玩', '玩', '碗', '弯', '园', '亮', '圆']
None 0
('身为学生，教室是最好的游乐场，因为一天可以利用的时间一半都在教室，从桌椅到门窗，从黑板到垃圾桶，都可以玩玩、欣赏。', 0.9998973608016968, '玩玩', 2, 19)

开始处理连续错误： 我喜欢秋天的风，也喜欢秋天的雨，不管是中秋月圆，还是牛郎与织女的微美故事。都深深的烧印在我的脑海中。秋天虽然有时令人感到凄凉，但仔细一看，倒发现其中也孕育著许多生命。秋天的风呼呼吹来，它将带著我的思念传到那充满欢笑声的小学。
predict_text: 我喜欢秋天的风，也喜欢秋天的雨，不管是中秋月圆，还是牛郎与织女的唯美故事。都深深地烙印在我的脑海中。秋天虽然有时令人感到凄凉，但仔细一看，倒发现其中也孕育著许多生命。秋天的风呼呼吹来，它将带著我的思念传到那充满欢笑声的小学。
['的', '地', '的', '得', '德', '地', '个', '到'] ['烧', '烙', '缫', '韶', '邵', '烫', '嗾', '铙']
None 0
('我喜欢秋天的风，也喜欢秋天的雨